In [5]:
!pip install gensim==4.3.1

  Using cached gensim-4.3.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (26.5 MB)
  Using cached scipy-1.10.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (34.5 MB)
  Using cached numpy-1.24.2-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
  Using cached smart_open-6.3.0-py3-none-any.whl (56 kB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pathy 0.6.2 requires smart-open<6.0.0,>=5.2.1, but you have smart-open 6.3.0 which is incompatible.
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [1]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
import pandas as pd
import numpy as np
from sklearn import svm
import random
from gensim.models import Word2Vec
import gensim.downloader as api
import string
import re
from nltk.tokenize import word_tokenize
from sklearn.impute import SimpleImputer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
nltk.download('vader_lexicon')
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


# iPhone 11

In [30]:
df = pd.read_csv("/work/iphone12-Apple.csv")

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   country            20 non-null     object
 1   countryCode        10 non-null     object
 2   date               20 non-null     object
 3   isVerified         20 non-null     bool  
 4   position           20 non-null     int64 
 5   productAsin        20 non-null     object
 6   ratingScore        20 non-null     int64 
 7   reviewDescription  20 non-null     object
 8   reviewImages/0     20 non-null     object
 9   reviewImages/1     10 non-null     object
 10  reviewReaction     20 non-null     object
 11  reviewTitle        20 non-null     object
 12  reviewUrl          20 non-null     object
 13  reviewedIn         20 non-null     object
 14  variant            19 non-null     object
dtypes: bool(1), int64(2), object(12)
memory usage: 2.3+ KB


In [32]:
df

,country,countryCode,date,isVerified,position,productAsin,ratingScore,reviewDescription,reviewImages/0,reviewImages/1,reviewReaction,reviewTitle,reviewUrl,reviewedIn,variant
0,United States,NaN,2020-11-09,True,1,B08L5P7DYY,4,Think this is the one great phone in world,https://images-na.ssl-images-amazon.com/images...,NaN,77 people found this helpful,Think this is the one great phone in world,https://www.amazon.com/gp/customer-reviews/R1R...,"Reviewed in the United States on November 9, 2020",Size: 128GBColor: Black
1,United States,NaN,2021-03-20,True,2,B08L5P7DYY,5,"The phone is great but, very limited use with ...",https://images-na.ssl-images-amazon.com/images...,NaN,15 people found this helpful,Nice phone,https://www.amazon.com/gp/customer-reviews/R3G...,"Reviewed in the United States on March 20, 2021",Size: 256GBColor: Green
2,United States,NaN,2021-01-16,True,3,B08L5P7DYY,5,very easy set up love my new phone im coming f...,https://images-na.ssl-images-amazon.com/images...,NaN,23 people found this helpful,easy set up love it,https://www.amazon.com/gp/customer-reviews/R2S...,"Reviewed in the United States on January 16, 2021",Size: 64GBColor: White
3,United States,NaN,2020-11-12,True,4,B08L5P7DYY,5,The phone is a definite upgrade from last year...,https://images-na.ssl-images-amazon.com/images...,NaN,79 people found this helpful,If your into OLED and HDR over iphone 11 u sho...,https://www.amazon.com/gp/customer-reviews/RK3...,"Reviewed in the United States on November 12, ...",Size: 128GBColor: Blue
4,United States,NaN,2021-01-17,True,5,B08L5P7DYY,5,Excellent phone for the price. It exceeded my ...,https://images-na.ssl-images-amazon.com/images...,NaN,23 people found this helpful,Best phone for the buck.,https://www.amazon.com/gp/customer-reviews/R2O...,"Reviewed in the United States on January 17, 2021",Size: 256GBColor: Blue
5,United States,NaN,2022-02-14,True,6,B08L5P7DYY,1,I return this item,https://images-na.ssl-images-amazon.com/images...,NaN,3 people found this helpful,I return this item,https://www.amazon.com/gp/customer-reviews/R2F...,"Reviewed in the United States on February 14, ...",Size: 256GBColor: Purple
6,United States,NaN,2021-03-17,True,7,B08L5P7DYY,1,Product was scratched up. Worse claims to have...,https://images-na.ssl-images-amazon.com/images...,NaN,34 people found this helpful,Not worth the price.,https://www.amazon.com/gp/customer-reviews/R1P...,"Reviewed in the United States on March 17, 2021",Size: 256GBColor: White
7,United States,NaN,2023-01-13,False,8,B08L5P7DYY,5,I don't know,https://images-na.ssl-images-amazon.com/images...,NaN,One person found this helpful,Not let it break the screen,https://www.amazon.com/gp/customer-reviews/R9C...,"Reviewed in the United States on January 13, 2023",Size: 128GBColor: Blue
8,United States,NaN,2022-07-09,False,9,B08L5P7DYY,4,Not very pleased but making do,https://images-na.ssl-images-amazon.com/images...,NaN,One person found this helpful,No better than old one Slow !,https://www.amazon.com/gp/customer-reviews/R22...,"Reviewed in the United States on July 9, 2022",Size: 64GBColor: (PRODUCT)RED
9,United States,NaN,2020-11-17,False,10,B08L5P7DYY,4,I have been using iPhones since I was in seven...,https://images-na.ssl-images-amazon.com/images...,NaN,32 people found this helpful,The iPhone 12 overall,https://www.amazon.com/gp/customer-reviews/RJP...,"Reviewed in the United States on November 17, ...",Size: 256GBColor: Black


In [33]:

df.drop(labels=['countryCode', 'country', 'date', 'isVerified', 'reviewImages/0', 'reviewReaction', 'reviewTitle', "reviewUrl", "reviewedIn", 'variant', 'reviewImages/1', 'productAsin'], axis=1)

,position,ratingScore,reviewDescription
0,1,4,Think this is the one great phone in world
1,2,5,"The phone is great but, very limited use with ..."
2,3,5,very easy set up love my new phone im coming f...
3,4,5,The phone is a definite upgrade from last year...
4,5,5,Excellent phone for the price. It exceeded my ...
5,6,1,I return this item
6,7,1,Product was scratched up. Worse claims to have...
7,8,5,I don't know
8,9,4,Not very pleased but making do
9,10,4,I have been using iPhones since I was in seven...


In [34]:
df.dropna(inplace=True)

In [35]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = re.sub(r'\@\w+', '', text)
    text = re.sub(r'\#\w+', '', text)
    text = re.sub(r'\d+', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = text.strip()
    return text

In [36]:
# tokenization
nltk.download('punkt')
df['clean_text'] = df['reviewDescription'].apply(clean_text)
df['tokens'] = df['clean_text'].apply(word_tokenize)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [37]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
df['filtered_tokens'] = df['tokens'].apply(lambda x: [word for word in x if word not in stop_words])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [38]:
# convert the list of tokens back to text
df['clean_text'] = df['filtered_tokens'].apply(lambda x: ' '.join(x))

## Sentiment Analysis

In [39]:
sid = SentimentIntensityAnalyzer()
df['sentiment'] = df['clean_text'].apply(lambda x: sid.polarity_scores(x)['compound'])

In [40]:
df['sentiment'] = df['sentiment'].apply(lambda x: 'positive' if x > 0 else 'negative')

In [41]:
def get_sentiment_score(text):
    sentiment_scores = sid.polarity_scores(text)
    return sentiment_scores['compound']

df['sentiment_score'] = df['clean_text'].apply(get_sentiment_score)

In [42]:
df

,country,countryCode,date,isVerified,position,productAsin,ratingScore,reviewDescription,reviewImages/0,reviewImages/1,reviewReaction,reviewTitle,reviewUrl,reviewedIn,variant,clean_text,tokens,filtered_tokens,sentiment,sentiment_score
10,United States,🇺🇸,2022-04-07,False,11,B08L5P7DYY,5,iPhone 12 is by far the most user-friendly sma...,https://images-na.ssl-images-amazon.com/images...,https://images-na.ssl-images-amazon.com/images...,One person found this helpful,Two Thumbs Up,https://www.amazon.com/gp/customer-reviews/RIN...,"Reviewed in the United States 🇺🇸 on April 7, 2022",Size: 128GBColor: Blue,iphone far userfriendly smartphone owned sleek...,"[iphone, is, by, far, the, most, userfriendly,...","[iphone, far, userfriendly, smartphone, owned,...",positive,0.9491
11,United States,🇺🇸,2020-11-09,False,12,B08L5P7DYY,4,The biggest selling point of the new iPhone is...,https://images-na.ssl-images-amazon.com/images...,https://images-na.ssl-images-amazon.com/images...,22 people found this helpful,"5G is practically USELESS, other than that gre...",https://www.amazon.com/gp/customer-reviews/RFW...,Reviewed in the United States 🇺🇸 on November 9...,Size: 64GBColor: Blue,biggest selling point new iphone g backstory l...,"[the, biggest, selling, point, of, the, new, i...","[biggest, selling, point, new, iphone, g, back...",positive,0.9877
12,United States,🇺🇸,2021-04-01,False,13,B08L5P7DYY,1,At my job I have plenty of opportunity to reco...,https://images-na.ssl-images-amazon.com/images...,https://images-na.ssl-images-amazon.com/images...,14 people found this helpful,Apple is not an ethical company.,https://www.amazon.com/gp/customer-reviews/R4P...,"Reviewed in the United States 🇺🇸 on April 1, 2021",Size: 256GBColor: Green,job plenty opportunity recommend phone custome...,"[at, my, job, i, have, plenty, of, opportunity...","[job, plenty, opportunity, recommend, phone, c...",negative,-0.4864
13,United States,🇺🇸,2021-02-09,False,14,B08L5P7DYY,3,The main reason I am giving 3 stars is because...,https://images-na.ssl-images-amazon.com/images...,https://images-na.ssl-images-amazon.com/images...,6 people found this helpful,Missing the AC adapter,https://www.amazon.com/gp/customer-reviews/R17...,Reviewed in the United States 🇺🇸 on February 9...,Size: 64GBColor: Blue,main reason giving stars apple continues cut a...,"[the, main, reason, i, am, giving, stars, is, ...","[main, reason, giving, stars, apple, continues...",positive,0.6808
14,United States,🇺🇸,2021-04-05,False,15,B08L5P7DYY,1,I have always loved my iphones and I have 10 i...,https://images-na.ssl-images-amazon.com/images...,https://images-na.ssl-images-amazon.com/images...,20 people found this helpful,Worse Phone ever,https://www.amazon.com/gp/customer-reviews/RKD...,"Reviewed in the United States 🇺🇸 on April 5, 2021",Size: 64GBColor: Black,always loved iphones iphones plan always loved...,"[i, have, always, loved, my, iphones, and, i, ...","[always, loved, iphones, iphones, plan, always...",positive,0.1808
16,United States,🇺🇸,2021-06-09,False,17,B08L5P7DYY,3,My wife talked me into getting my first iPhone...,https://images-na.ssl-images-amazon.com/images...,https://images-na.ssl-images-amazon.com/images...,3 people found this helpful,Be careful setting up your Apple icloud account,https://www.amazon.com/gp/customer-reviews/RHH...,"Reviewed in the United States 🇺🇸 on June 9, 2021",Size: 64GBColor: Green,wife talked getting first iphone taking long t...,"[my, wife, talked, me, into, getting, my, firs...","[wife, talked, getting, first, iphone, taking,...",positive,0.6131
17,United States,🇺🇸,2020-10-29,False,18,B08L5P7DYY,3,iPhone is great but not really white. Is an of...,https://images-na.ssl-images-amazon.com/images...,https://images-na.ssl-images-amazon.com/images...,28 people found this helpful,iPhone great But not white. More eggshell,https://www.amazon.com/gp/customer-reviews/RKQ...,Reviewed in the United States 🇺🇸 on October 29...,Size: 64GBColor: White,iphone great really white 

In [43]:

df['review_rating'] = df['ratingScore'].astype(float)

In [44]:
positive_threshold = 4.0
negative_threshold = 2.0
positive_text = df[df['review_rating'] >= positive_threshold]
negative_text = df[df['review_rating'] <= negative_threshold]

In [45]:
print('Positive Reviews')
print(positive_text['clean_text'])

Positive Reviews
10    iphone far userfriendly smartphone owned sleek...
11    biggest selling point new iphone g backstory l...
18    ’ sure negative reviews granted upgraded mini ...
Name: clean_text, dtype: object


In [46]:
print('Negative Reviews:')
print(negative_text['clean_text'])

Negative Reviews:
12    job plenty opportunity recommend phone custome...
14    always loved iphones iphones plan always loved...
19    product came continuous software issues produc...
Name: clean_text, dtype: object


### lowest sentiment text

In [47]:
lowest_sentiment_row = df.sort_values(by='sentiment_score').iloc[0]
print('Review text:', lowest_sentiment_row['clean_text'])
print('Sentiment score:', lowest_sentiment_row['sentiment_score'])

Review text: job plenty opportunity recommend phone customers know everything know it— far functionality— highly recommend besides fact possibly worst looking iphone thus far obviously close second iphone die hard fan help get get used iphone ’ hard go back learning curve ’ never iphone save even one person— time— making switch world better place start using iphone first company ethical making phone— close human slavery streamlined manufacturing plants overseas get away everything unethical usa sold car years made new model would illegal wrong go garage ruin engine knowing buy new car yet apple time yes lost lawsuits still get away it— especially us less frequently europe personally experienced current iphone apple wants upgrade want upgrade apple update takes away service find extremely unethical literally pay phone coverage phone since millisecond new update shows service someone identical phone sued exact reason apple told production number phone fell outside reimbursements secondly

### highest sentiment text

In [48]:
highest_sentiment_row = df.sort_values('sentiment_score', ascending=False).iloc[0]
print('Review text:', highest_sentiment_row['clean_text'])
print('Sentiment score:', highest_sentiment_row['sentiment_score'])


Review text: biggest selling point new iphone g backstory live fairly large suburban city near east los angeles population density ive looking phone would offer good cellular internet away wifi im tmobile network around area im getting around mbps mbsp worse lte traveled outside major metro areas actually change service lte get usable internet allother iphone works like noticed bit snappier previous xs max particularly snapping home opening new applications night mode also something else cool feature one surprising improvement earpods bluetooth devices sound better may due new bluetooth technology new iphone also noticeably lighter previous xs max felt like brick small improves includes brighter display dynamic contrastthe biggest cool feature think bought back magsafe see myriads docks carmounts photography gadgets phone cases wallets chargers stands tripods gaming controller portable battery packs etc take advantage featuresome negatives ive noticed speakers maybe weight saving dont 

In [49]:
def clean_text(reviewDescription):
    if isinstance(reviewDescription, str):
        text = review_text.lower() # Convert to lowercase
        tokens = word_tokenize(text) # Tokenize the text
        tokens = [word for word in tokens if word.isalpha()] # Remove punctuation
        tokens = [word for word in tokens if word not in stop_words] # Remove stop words
        return tokens
    else:
        return ""

In [50]:
data =  positive_text + negative_text
labels = ['Positive'] * len(positive_text) + ['Negative']

In [51]:
data = [clean_text(text) for text in data['reviewDescription']]

vectorizer = TfidfVectorizer()
text_data = vectorizer.fit_transform(df['clean_text'].values.astype('U')) # Convert to Unicode string

labels = df['reviewDescription'].values

In [65]:

train_data, test_data, train_labels, test_labels = train_test_split(text_data, labels, test_size=0.2, random_state=42)

imputer = SimpleImputer(strategy='mean')
train_data = imputer.fit_transform(train_data)


## Logistic Regression

In [61]:
model = LogisticRegression()
model.fit(train_data, train_labels)

test_predictions = model.predict(test_data)

test_accuracy = accuracy_score(test_labels, test_predictions)
print("Test accuracy:", test_accuracy)

Test accuracy: 0.0


## Naive Bayes 

In [54]:
model = MultinomialNB()
model.fit(train_data, train_labels)

test_predictions = model.predict(test_data)

test_accuracy = accuracy_score(test_labels, test_predictions)
print("Test accuracy:", test_accuracy)

Test accuracy: 0.0


## Random Forest 

In [55]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(train_data, train_labels)

y_pred = model.predict(test_data)

test_accuracy = accuracy_score(test_labels, test_predictions)
print("Test accuracy:", test_accuracy)

Test accuracy: 0.0


# oversampled

In [62]:
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import GridSearchCV
ros = RandomOverSampler(random_state=42)
train_data, train_labels = ros.fit_resample(train_data, train_labels)

In [66]:
logistic = LogisticRegression(max_iter=1000)
param_grid = {'C': [0.01, 0.1, 1, 10], 'penalty': ['l1', 'l2']}
grid_search = GridSearchCV(logistic, param_grid, cv=5, scoring='f1_macro')

# Fit the GridSearchCV object on the training data
grid_search.fit(train_data, train_labels)

# Get the best estimator from the GridSearchCV object
best_model = grid_search.best_estimator_

# Use the best estimator to predict the test labels
test_predictions = best_model.predict(test_data)

# Calculate the accuracy of the test predictions
test_accuracy = accuracy_score(test_labels, test_predictions)
print("Test accuracy:", test_accuracy)


ValueError: n_splits=5 cannot be greater than the number of members in each class.

In [0]:
model = MultinomialNB()
model.fit(train_data, train_labels)

test_predictions = model.predict(test_data)

test_accuracy = accuracy_score(test_labels, test_predictions)
print("Test accuracy:", test_accuracy)

In [0]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(train_data, train_labels)

y_pred = model.predict(test_data)

test_accuracy = accuracy_score(test_labels, test_predictions)
print("Test accuracy:", test_accuracy)

In [0]:
def clean_text(text):
    if isinstance(text, str):
        text = text.lower()
        tokens = text.split()
        tokens = [word.strip('.,!?()[]{}') for word in tokens]
        stop_words = set(['the', 'and', 'a', 'an', 'in', 'of', 'to', 'is', 'for'])
        tokens = [word for word in tokens if word not in stop_words]
        return ' '.join(tokens)
    else:
        return ""

In [0]:
selected_word = input('Enter a word to search for: ')
selected_reviews = df[df['clean_text'].str.contains(selected_word)]

sid = SentimentIntensityAnalyzer()
selected_reviews['sentiment_score'] = selected_reviews['review_text'].apply(lambda x: sid.polarity_scores(x)['compound'])
positive_reviews = selected_reviews[selected_reviews['sentiment_score'] > 0]
negative_reviews = selected_reviews[selected_reviews['sentiment_score'] <= 0]

# Print out the selected reviews by sentiment category
print(f"Positive reviews containing the word '{selected_word}':")
for review_text in positive_reviews['review_text']:
    print(review_text)

print(f"\nNegative reviews containing the word '{selected_word}':")
for review_text in negative_reviews['review_text']:
    print(review_text)

In [0]:
positive_selected_reviews = selected_reviews[(selected_reviews['sentiment_score'] > 0)]
num_positive_selected = len(positive_selected_reviews[positive_selected_reviews['clean_text'].str.contains(selected_word)])
num_total_selected = len(selected_reviews[selected_reviews['clean_text'].str.contains(selected_word)])
positive_percentage = num_positive_selected / num_total_selected * 100                           

# Print out the percentage of positive reviews containing the selected word
print(f"Percentage of positive reviews containing the word '{selected_word}': {positive_percentage:.2f}%")

In [0]:
negative_selected_reviews = selected_reviews[(selected_reviews['sentiment_score'] <= 0)]
num_negative_selected = len(negative_selected_reviews[negative_selected_reviews['clean_text'].str.contains(selected_word)])
num_total_selected = len(selected_reviews[selected_reviews['clean_text'].str.contains(selected_word)])
negative_percentage = num_negative_selected / num_total_selected * 100                           

# Print out the percentage of negative reviews containing the selected word
print(f"Percentage of negative reviews containing the word '{selected_word}': {negative_percentage:.2f}%")

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=320e99cb-8487-44b0-a3ee-3aa3add03759' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>